# HDL PRS example

Here we show an example of our pipeline for HDL PRS on UK Biobank samples.

## Data used

### Reference panel

Obtained via `download_1000G()` in `bigsnpr`. 

Including 503 (mostly unrelated) European individuals and ~1.7M SNPs in common with either HapMap3 or the UK Biobank. Classification of European population can be found at [IGSR](https://www.internationalgenome.org/category/population/). European individuals ID are from [IGSR data portal](https://www.internationalgenome.org/data-portal/sample).

### GWAS summary statistics data

From MVP. We have the original GWAS summary data as well as multivariate posterior estimate of HDL effects using [mashr](https://github.com/stephenslab/mashr). In brief, we have two versions of summary statistics (effect estimates) for HDL.

### Target test data: UK biobank

We select randomly from UK Biobank 2000 individuals with covariates and HDL phenotype (medication adjusted, inverse normalized). Their genotypes are extracted. See `UKB.QC.*` PLINK file bundle. 

### PRS Models

Auto model runs the algorithm for 30 different $p$ (the proportion of causal variants) values range from 10e-4 to 0.9, and heritability $h^2$ from LD score regression as initial value.

Grid model tries a grid of parameters $p$, ranges from 0 to 1 and three $h^2$ which are 0.7/1/1.4 times of initial $h^2$ estimated by LD score regression.

## Test genotype data preparation

Use `awk` select columns in phenotypes file saved to traits file `UKB.hdl.cov` and covariates file `UKB.ind.cov`.

In [1]:
fam_UKB <- read.table("ukbiobank/UKB.fam", header = F, stringsAsFactors = F)
colnames(fam_UKB)=c("FID","IID","paternal.ID","maternal.ID","sex","affection")
covariates <- read.table("ukbiobank/UKBCauc_cholesterolandbloodpressurefields_inverseranknorm_covariatesage_sex_alcohol_smokingpackyears_foranalysis", header = T, stringsAsFactors = F)
suppressMessages(library(tidyverse))

set.seed(2021)
covariate = covariates %>%
    drop_na() %>%
    filter(FID %in% sample(FID,2000))
    
covariate = covariate[order(covariate[,1]),]

fam_UKB = fam_UKB %>% filter(FID %in% covariate$FID)
fam_UKB = fam_UKB[order(covariate[,1]),]
colnames(fam_UKB)<-NULL

write.table(covariate, file = "ukbiobank/UKB.cov", sep = " ", 
            row.names = F, col.names = T,)
write.table(fam_UKB, file = "ukbiobank/UKB.QC.fam", sep = " ", 
            row.names = FALSE, col.names = FALSE)

In [ ]:
cd ukbiobank
awk '{print $3, $4}' UKB.cov > UKB.ind.cov
awk '{print $14}' UKB.cov > UKB.hdl.cov
cd ..

We only want to focus on 2000 selected samples in UKB as target data. Below we extract this subset,

In [ ]:
sos run ldpred.ipynb snp_qc \
    --cwd output \
    --genoFile  /path/to/full/UKB.bed \
    --keep-samples /path/to/UKB.QC.fam \
    --name 2000_subset

## Summary statistics data preparation

In [3]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.0.5     ✔ dplyr   1.0.2
✔ tidyr   1.0.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [14]:
format_data <- function(sums, chrpos, prefix) {
  sums = inner_join(sums, chrpos, by = c("chr","pos")) %>% select(chr, pos, rsid, a0, a1, everything())
  
  print("inner join completed")
  
  sumstats_hdl  = sums %>% 
    select(chr, pos, rsid, a0,a1, hdl, hdl_se) %>% 
    rename(beta = hdl,
          beta_se = hdl_se) %>% 
    mutate(n_eff = 200000)
  
  print("finish hdl")
  
  sumstats_ldl  = sums %>% 
   select(chr, pos, rsid, a0, a1, ldl, ldl_se) %>% 
    rename(beta = ldl,
          beta_se = ldl_se) %>% 
    mutate(n_eff = 200000)
  
  print("finish ldl")

  sumstats_tc  = sums %>% 
    select(chr, pos, rsid, a0,a1, tc, tc_se) %>% 
    rename(beta = tc,
          beta_se = tc_se) %>% 
    mutate(n_eff = 200000)
  
  print("finish tc")

  sumstats_tg  = sums %>% 
    select(chr, pos, rsid, a0,a1, tg, tg_se) %>% 
    rename(beta = tg,
          beta_se = tg_se) %>% 
    mutate(n_eff = 200000)
  
  print("finish tg")
  
  print("data ready to save")
    saveRDS(sumstats_hdl, file = paste0(prefix, "_hdl.rds"))
    saveRDS(sumstats_ldl, file = paste0(prefix, "_ldl.rds"))
    saveRDS(sumstats_tc, file = paste0(prefix, "_tc.rds"))
    saveRDS(sumstats_tg, file = paste0(prefix, "_tg.rds"))  
}

In [4]:
chr_pos = bigreadr::fread2("mvpdata/chr_pos_allele2_lfsr.txt")
head(chr_pos)

chr_pos = chr_pos %>%
  extract(V1, into = c("chr", "pos"), regex = "([0-9]+)(\\:[0-9]+)") %>% 
  mutate(chr = as.numeric(chr),
         pos = as.numeric(str_replace(pos, ":",""))) %>% 
  rename(a0 = Allele1,
         a1 = Allele2,
         rsid = rsID) %>% 
  select(chr,pos,rsid,a0,a1)
head(chr_pos)

Warning message in data.table::fread(input, ..., data.table = data.table, nThread = nThread):
“Detected 7 column names but the data has 8 columns (i.e. invalid file). Added 1 extra default column name for the first column which is guessed to be row names or an index. Use setnames() afterwards if this guess is not correct, or fix the file write command that created the file to create a valid file.”


,V1,Allele1,Allele2,rsID,hdl,ldl,tg,tc
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,1:748332,T,C,rs182373484,0.5109300,0.3644157,0.5689165,0.3591687
2,1:751343,A,T,rs28544273,0.4295461,0.5213810,0.4557329,0.5505538
3,1:751756,T,C,rs143225517,0.4322768,0.5181581,0.4569321,0.5463038
4,1:752566,A,G,rs3094315,0.5774074,0.6145746,0.5934880,0.6117367
5,1:752721,A,G,rs3131972,0.5833319,0.5641637,0.5950461,0.5602104
6,1:753405,A,C,rs3115860,0.4092361,0.4824255,0.4297362,0.5109667


,chr,pos,rsid,a0,a1
,<dbl>,<dbl>,<chr>,<chr>,<chr>
1,1,748332,rs182373484,T,C
2,1,751343,rs28544273,A,T
3,1,751756,rs143225517,T,C
4,1,752566,rs3094315,A,G
5,1,752721,rs3131972,A,G
6,1,753405,rs3115860,A,C


In [6]:
raw_beta = readRDS("mvpdata/zmash_raw_univariate_MVP.rds")
raw_beta_se = bigreadr::fread2("mvpdata/Merged_MVP_Full_se_raw.txt")

Warning message in data.table::fread(input, ..., data.table = data.table, nThread = nThread):
“Detected 5 column names but the data has 6 columns (i.e. invalid file). Added 1 extra default column name for the first column which is guessed to be row names or an index. Use setnames() afterwards if this guess is not correct, or fix the file write command that created the file to create a valid file.”


In [8]:
raw_beta_se = raw_beta_se %>% 
  data.frame() %>% 
  rename(marker = m.Marker)
raw_beta = raw_beta %>% 
  data.frame() %>% 
  mutate(marker = rownames(raw_beta))

sumstats = inner_join(raw_beta_se,raw_beta, by = 'marker') %>% 
  select(marker, hdl,HDLSe,ldl,LDLSe, tg, TGSe, tc, TCSe)

colnames(sumstats) = c("marker", "hdl_z", "hdl_se", "ldl_z", "ldl_se", "tg_z", "tg_se", "tc_z", "tc_se")

sumstats = sumstats %>%
  extract(marker, into = c("chr", "pos"), regex = "([0-9]+)(\\:[0-9]+)") %>% 
  mutate(chr = as.numeric(chr),
         pos = as.numeric(str_replace(pos, ":","")))

In [ ]:
# adjust the input GWAS summary stats to get bhat from z-scores
sumstats = sumstats %>% 
  mutate(hdl = hdl_z*hdl_se,
        ldl = ldl_z*ldl_se,
        tc = tc_z*tc_se,
        tg = tg_z*tg_se) %>% 
  select(-hdl_z,-ldl_z, -tc_z, -tg_z)

In [16]:
format_data(sumstats, chr_pos, "mvpdata/gwas")

[1] "inner join completed"
[1] "finish hdl"
[1] "finish ldl"
[1] "finish tc"
[1] "finish tg"
[1] "data ready to save"


In [17]:
pos_raw_beta = bigreadr::fread2("mvpdata/MVP_all_beta_posterior_beta.txt")
pos_raw_beta_se = bigreadr::fread2("mvpdata/posterior_beta_se.txt")

colnames(pos_raw_beta)=c("marker", "hdl","ldl","tg","tc")
colnames(pos_raw_beta_se)=c("marker", "hdl_se","ldl_se","tg_se","tc_se")

Warning message in data.table::fread(input, ..., data.table = data.table, nThread = nThread):
“Detected 4 column names but the data has 5 columns (i.e. invalid file). Added 1 extra default column name for the first column which is guessed to be row names or an index. Use setnames() afterwards if this guess is not correct, or fix the file write command that created the file to create a valid file.”
Warning message in data.table::fread(input, ..., data.table = data.table, nThread = nThread):
“Detected 4 column names but the data has 5 columns (i.e. invalid file). Added 1 extra default column name for the first column which is guessed to be row names or an index. Use setnames() afterwards if this guess is not correct, or fix the file write command that created the file to create a valid file.”


In [18]:
head(pos_raw_beta)

,marker,hdl,ldl,tg,tc
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,1:100000012,2.460009e-03,-0.0001055433,-0.0019129145,-0.0001208528
2,1:10000006,-6.929132e-05,0.0018424284,-0.0001815899,0.0014059207
3,1:100000507,-8.461508e-03,0.0171136365,0.0144205721,0.0179276152
4,1:100000827,3.042258e-03,0.0001911716,-0.0024015024,0.0001091159
5,1:100000843,-1.196533e-03,-0.0003462751,0.0011000920,-0.0002095452
6,1:100000948,1.142429e-02,0.0034483668,-0.0070852939,0.0034940914


In [19]:
head(pos_raw_beta_se)

,marker,hdl_se,ldl_se,tg_se,tc_se
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,1:100000012,0.002291870,0.001469390,0.001952622,0.001596889
2,1:10000006,0.014067514,0.011210222,0.014404394,0.012153967
3,1:100000507,0.027183021,0.028191475,0.029768057,0.029389961
4,1:100000827,0.002307946,0.001522781,0.002027389,0.001655077
5,1:100000843,0.002689326,0.002029852,0.002632001,0.002180022
6,1:100000948,0.014627076,0.010156359,0.012015341,0.010786769


In [20]:
pos_raw_beta_se = pos_raw_beta_se %>% 
  data.frame()
pos_raw_beta = pos_raw_beta %>% 
  data.frame()

In [21]:
sumstats_pos = inner_join(pos_raw_beta_se,pos_raw_beta, by = 'marker') %>% 
  select(marker, hdl,hdl_se,ldl,ldl_se, tg, tg_se, tc, tc_se)

head(sumstats_pos)

colnames(sumstats_pos) = c("marker", "hdl", "hdl_se", "ldl", "ldl_se", "tg", "tg_se", "tc", "tc_se")

sumstats_pos = sumstats_pos %>%
  extract(marker, into = c("chr", "pos"), regex = "([0-9]+)(\\:[0-9]+)") %>% 
  mutate(chr = as.numeric(chr),
         pos = as.numeric(str_replace(pos, ":","")))

,marker,hdl,hdl_se,ldl,ldl_se,tg,tg_se,tc,tc_se
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1:100000012,2.460009e-03,0.002291870,-0.0001055433,0.001469390,-0.0019129145,0.001952622,-0.0001208528,0.001596889
2,1:10000006,-6.929132e-05,0.014067514,0.0018424284,0.011210222,-0.0001815899,0.014404394,0.0014059207,0.012153967
3,1:100000507,-8.461508e-03,0.027183021,0.0171136365,0.028191475,0.0144205721,0.029768057,0.0179276152,0.029389961
4,1:100000827,3.042258e-03,0.002307946,0.0001911716,0.001522781,-0.0024015024,0.002027389,0.0001091159,0.001655077
5,1:100000843,-1.196533e-03,0.002689326,-0.0003462751,0.002029852,0.0011000920,0.002632001,-0.0002095452,0.002180022
6,1:100000948,1.142429e-02,0.014627076,0.0034483668,0.010156359,-0.0070852939,0.012015341,0.0034940914,0.010786769


In [22]:
format_data(sumstats_pos, chr_pos, "mvpdata/mash")

[1] "inner join completed"
[1] "finish hdl"
[1] "finish ldl"
[1] "finish tc"
[1] "finish tg"
[1] "data ready to save"


At this point files on the disk should be:

In [23]:
tree

.
├── 1000G.EUR
│   ├── 1000G.EUR.bed
│   ├── 1000G.EUR.bim
│   └── 1000G.EUR.fam
├── mvpdata
│   ├── chr_pos_allele2_lfsr.txt
│   ├── gwas_hdl.rds
│   ├── gwas_ldl.rds
│   ├── gwas_tc.rds
│   ├── gwas_tg.rds
│   ├── mash_hdl.rds
│   ├── mash_ldl.rds
│   ├── mash_tc.rds
│   ├── mash_tg.rds
│   ├── Merged_MVP_Full_se_raw.txt
│   ├── MVP_all_beta_posterior_beta.txt
│   ├── posterior_beta_se.txt
│   └── zmash_raw_univariate_MVP.rds
└── ukbiobank
    ├── UKB.cov
    ├── UKB.hdl.cov
    ├── UKB.ind.cov
    └── UKB.QC.fam

3 directories, 20 files


## Step 1: QC and select SNPs shared by summary statistics and target data

In [29]:
sos run ldpred.ipynb snp_qc \
    --cwd output \
    --genoFile  1000G.EUR/1000G.EUR.bed

INFO: Running basic QC filters: Filter SNPs and select individuals
INFO: basic QC filters is completed.
INFO: basic QC filters output:   /home/gw/tmp/18-May-2021/output/cache/1000G.EUR.output.filtered.bed
INFO: Workflow snp_qc (ID=w0fc7d31d81e2bd2c) is executed successfully with 1 completed step.


In [7]:
sos run ldpred.ipynb common_snp \
    --outpath res-data \
    --testpath ukbiobank \
    --stat_snp mvpdata/sumstats_hdl.snplist \
    --ref_snp 1000G/1000G.QC.snplist \
    --test_snp ukbiobank/UKB.QC.snplist \
    --summstats_file mvpdata/sumstats_hdl.rds \
    --sub_stats mvpdata/sumstats_hdl.SUB.rds

INFO: Running common_snp: 2004l
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
[1] "There are  409486  common SNPs."
INFO: common_snp is completed.
INFO: common_snp output:   mvpdata/sumstats_hdl.SUB.rds res-data/common.snplist
INFO: Workflow common_snp (ID=wda5082e8178fcf44) is executed successfully with 1 completed step.


: 1

## Step 2: Extract SNPs also in reference panel

In [9]:
sos run ldpred.ipynb subsets \
    --outpath res-data \
    --testpath ukbiobank \
    --bed_file 1000G/1000G.EUR.bed \
    --fam_file 1000G/1000G.EUR.fam \
    --snp_file res-data/common.snplist \
    --sub_bedfile 1000G/1000G.SUB.bed

INFO: Running subsets: 
PLINK v1.90b6.22 64-bit (16 Apr 2021)          www.cog-genomics.org/plink/1.9/
(C) 2005-2021 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to 1000G/1000G.SUB.log.
Options in effect:
  --bfile 1000G/1000G.EUR
  --extract res-data/common.snplist
  --keep 1000G/1000G.EUR.fam
  --make-bed
  --out 1000G/1000G.SUB

8192 MB RAM detected; reserving 4096 MB for main workspace.
1664852 variants loaded from .bim file.
503 people (240 males, 263 females) loaded from .fam.
--extract: 409486 variants remaining.
--keep: 503 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 503 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
409486 variants and 503 people pass filters and QC.
Note: No phenotypes

: 1

Totally 31566 varients

    ./plink \
        --bfile 1000G/1000G.EUR \
        --keep 1000G/1000G.EUR.fam \
        --extract res-data/common.snplist \
        --make-bed \
        --out 1000G/1000G.SUB


## Step 3: Harmonize alleles for shared SNPs

To handle major/minor allele, strand flips and consequently possible flips in sign for summary statistics.

In [ ]:
sos run ldpred.ipynb snp_match \
    --outpath res-data \
    --testpath ukbiobank \
    --ref_bfile 1000G/1000G.SUB.bed \
    --ref_file 1000G/1000G.SUB.rds \
    --summstats_file mvpdata/sumstats_hdl.SUB.rds \
    --n_eff 200000 \
    --test_snplist UKB.SUB.snplist

## Step 4: Summary statistics quality control (optional)

In this example this workflow seems to have dropped so SNPs that PRS no longer contributes. See results at the end of the document.

In [ ]:
sos run ldpred.ipynb QControl \
    --qc_in res-data/MatchedSnp.RData \
    --outpath res-data \
    --testpath ukbiobank \
    --test_snplist UKB.QC.SUB.snplist

## Step 5: Extract test target data

Subset the genotype data for randomly selected samples

In [ ]:
sos run ldpred.ipynb subsets \
    --outpath res-data \
    --testpath ukbiobank \
    --bed_file ukbiobank/UKB.bed \
    --fam_file ukbiobank/UKB.QC.fam \
    --snp_file ukbiobank/UKB.SUB.snplist \
    --sub_bedfile ukbiobank/UKB.SUB.bed

## Step 6: Calculate LD matrix (correlation matrix)

In [ ]:
sos run ldpred.ipynb LD \
    --outpath res-data \
    --testpath ukbiobank \
    --ld_in res-data/MatchedSnp.Rdata

## Step 7: Estimate posterior effect sizes and PRS

In [ ]:
sos run ldpred.ipynb load_testdata+inf_prs \
    --outpath res-data \
    --testpath ukbiobank \
    --inf_in res-data/LdMatrix.Rdata \
    --test_bfile ukbiobank/UKB.SUB.bed \
    --test_file ukbiobank/UKB.SUB.rds

In [ ]:
sos run ldpred.ipynb grid_prs \
    --outpath res-data \
    --testpath ukbiobank \
    --grid_in res-data/LdMatrix.Rdata \
    --test_bfile ukbiobank/UKB.SUB.bed \
    --cov_file ukbiobank/UKB.ind.cov \
    --trait_file ukbiobank/UKB.hdl.cov \
    --test_file ukbiobank/UKB.SUB.rds \
    --response continuous

In [ ]:
sos run ldpred.ipynb auto_prs \
    --outpath res-data \
    --testpath ukbiobank \
    --auto_in res-data/LdMatrix.Rdata \
    --test_bfile ukbiobank/UKB.SUB.bed \
    --test_file ukbiobank/UKB.SUB.rds

## Step 8: predict phenotypes

Null model: Traits ~ Sex + Age

In [5]:
sos run ldpred.ipynb null_phenopred \
    --outpath res-data \
    --testpath ukbiobank \
    --cov_file ukbiobank/UKB.ind.cov \
    --trait_file ukbiobank/UKB.hdl.cov \
    --response continuous

INFO: Running null_phenopred: l
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
Loading required package: bigstatsr
HDL_inverseranknorm ~ AGE + SEX
<environment: 0x7fdc0ec54d20>

Call:
lm(formula = ., data = data[train.ind, ])

Residuals:
    Min      1Q  Median      3Q     Max 
-2.7973 -0.6148 -0.0072  0.5943  3.2185 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) -0.083025   0.177526  -0.468   0.6401    
AGE          0.007225   0.003028   2.386   0.0171 *  
SEX         -0.857958   0.046395 -18.492   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.9257 on 1597 degrees of freedom
Multiple R-squared:  0.177,	Adjusted R-squared:  0.176 
F-statistic: 171.7 on 2 and 1597 DF,  p-value: < 2.2e-16

# A tibble: 1 x 3
  model         R2   MSE
  <chr>      <

: 1

Inf/grid/auto model: Traits ~ Sex + Age + PRS

In [4]:
sos run ldpred.ipynb inf_phenopred \
    --outpath res-data \
    --testpath ukbiobank \
    --cov_file ukbiobank/UKB.ind.cov \
    --trait_file ukbiobank/UKB.hdl.cov \
    --prs_file res-data/InfPred.Rdata \
    --mod_summary InfSummary.pdf \
    --model InfModel.Rdata \
    --response continuous

INFO: Running inf_phenopred: 4l
During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 
Loading required package: bigstatsr
HDL_inverseranknorm ~ PRS + AGE + SEX
<environment: 0x7fe452c226e0>

Call:
lm(formula = reg.formula, data = data[train.ind, ])

Residuals:
    Min      1Q  Median      3Q     Max 
-2.9595 -0.5773 -0.0030  0.5575  3.8241 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) -0.303484   0.166778  -1.820   0.0690 .  
PRS         -2.628765   0.174294 -15.082   <2e-16 ***
AGE          0.006512   0.002834   2.298   0.0217 *  
SEX         -0.839663   0.043436 -19.331   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.8663 on 1596 degrees of freedom
Multiple R-squared:  0.2797,	Adjusted R-squared:  0.2783 
F-statistic: 206.5 on 3 and 1596 DF,  p-val

: 1

In [ ]:
sos run ldpred.ipynb grid_phenopred \
    --outpath res-data \
    --testpath ukbiobank \
    --cov_file ukbiobank/UKB.ind.cov \
    --trait_file ukbiobank/UKB.hdl.cov \
    --prs_file res-data/GridPred.Rdata \
    --mod_summary GridSummary.pdf \
    --model GridModel.Rdata \
    --response continuous

In [ ]:
sos run ldpred.ipynb auto_phenopred \
    --outpath res-data \
    --testpath ukbiobank \
    --cov_file ukbiobank/UKB.ind.cov \
    --trait_file ukbiobank/UKB.hdl.cov \
    --prs_file res-data/AutoPred.Rdata \
    --mod_summary AutoSummary.pdf \
    --model AutoModel.Rdata \
    --response continuous

# Results

Following table shows adjusted R squared of HDL prediction model. QC is quality control in step 4.

|   Betas   | QC? |   Null  |   Inf   |   Grid  |   Auto  |
|:---------:|:---:|:-------:|:-------:|:-------:|:-------:|
|  Original | Yes | 0.2901 | 0.2899 | 0.2901  | 0.2897 |
|  Original |  No$^*$ | - | - |    -  |    -   |
| Posterior | Yes | 0.2901 | 0.2897 | 0.2901| 0.29 |
| Posterior |  No$^*$ |  -  |  - |    -    |    -   |

|   Betas   | QC? |   Null  |   Inf   |   Grid  |   Auto  |
|:---------:|:---:|:-------:|:-------:|:-------:|:-------:|
|  Original | Yes | 0.176 | 0.1756 | 0.1757  | 0.1755 |
|  Original |  No$^*$ | - | - |    -  |    -   |
| Posterior | Yes | 0.176 | 0.1756 | 0.1758| 0.1756 |
| Posterior |  No$^*$ |  0.176  |  0.2783 |    -    |    -   |

$*$: Take long very long time to get results (4hrs+)